# TIM Python Client - Tutorial - Setup

Welcome to this tutorial for using the Python Client for TIM. Before we get started with building models and predictions with TIM we will setup our working environment within the platform.

Make sure you have the latest version of the TIM Python Client installed. Please visit https://pypi.org/project/tim-client/ and https://docs.tangent.works/docs/TIM-Clients/Python-Client/Installation for more information.

To activate the client and use the client we will import the TIM library and other supporting libraries into this notebook. Attached to this tutorial you will find a JSON-file called tim_credentials.json. Please write your username and password into to JSON to be able to authenicate your connection to TIM.

In [18]:
import json
import os

In [4]:
import tim
tim_credentials = json.load(open('tim_credentials.json'))
client = tim.Tim(email=tim_credentials['email'],password=tim_credentials['password'])

The TIM Platform allows you to organize your work with TIM on different levels.
- __User groups__:  
User groups allow you to work on the same topics with the other users on your license. 
- __Workspaces__:  
Under each user group workspaces can be created to work on specific topics.  
On this level we will add and manage the datasets we want to analyze with TIM. 
- __Use Cases__:  
Under each workspace use cases can be created to analyze the dataset which exist within the workspace.  
Each dataset can be analyzed in multiple use cases. A use case does not manage multiple datasets.
- __Experiments__:  
Under each use case experiments can be created. On this level we make the distinction between forecasting and anomaly detection.  
The experiments will contain all the model and prediction jobs.  
 
The TIM Python Client is organized in two major fields: API Endpoint connection methods and Extended functions.
- __Endpoint methods__:  
The endpoint methods allow you to access each individual endpoint in the TIM REST API.  
You can visit the API here: https://tim-platform.tangent.works/api/v5/swagger-ui.html#/  
With these methods you have full flexibility to communicate directly will all capabilities of the TIM Platform.  
For example:

In [7]:
# client.forecasting.build_model()

This method registers a forecasting build model job in TIM. 

- __Extended functions__:  
The extended functions are built on top of the endpoint methods and combine several endpoints together to facilitate the work with TIM.  
With these functions you can quickly get started with TIM and leverage the API efficiently.  
For example:

In [8]:
# client.extended_forecasting.build_model()

This method can register and execute a forecasting build model job in TIM, check the status of the job and collect results back from TIM.  
This goes well beyond the endpoint method with the same name. More details about the use of these functions will follow in this tutorial.

## ------------------------------------------------ Workflow ------------------------------------------------

We will create a workflow in TIM to organize ourselves in this tutorial and make efficient use of the TIM Platform.  
This section will provide you with the necessary insights and capabilities to work on large scale and volume use cases with TIM.  
First, let's create a folder to store and keep trace of the id's of TIM objects (such as user groups and workspaces ...) we are going to create.

In [20]:
if not os.path.exists('tim_workflow'):
    os.makedirs('tim_workflow')

Secondly, let's collect your user_id from TIM.

In [12]:
users_details_user = client.users.details_user()
user_id = users_details_user['id']

### 1. User groups

Next we will create a user group called "Tutorial". For this we will use the method below and provide it with a user group configuration. The configuration needs to contain a name and one or more user id's. Set the ownership to "True" which allows you to make changes to the user group later if needed. You can also add a description if needed.

In [22]:
create_user_group_configuration = {
  "name": "Tutorial",
  "description": "This user group contains all work from the TIM Tutorials.",
  "users": [
    {
      "id": user_id,
      "isOwner": True
    }
  ]
}

Running the next method will create the configured user group within the TIM platform.

In [24]:
create_user_group_response = client.user_groups.create_user_group(configuration = create_user_group_configuration)
create_user_group_response

{'name': 'Tutorial',
 'createdBy': 'fa4b9fce-f6f6-4f07-bab1-5814a537c413',
 'id': '92d09891-9a8b-4b06-94d6-fe8c7c719bc3',
 'createdAt': '2023-04-06T09:26:13.392Z',
 'description': 'This user group contains all work from the TIM Tutorials.'}

We will save this JSON in the workflow directory for later use and already collect the user_group_id to continue.

In [27]:
json.dump(create_user_group_response,open("./tim_workflow/tim_user_group.json", "w"))
user_group_id = create_user_group_response['id']

Make sure not to run the create_user_group method again since that will create another user group with the same name.  
To check if the "Tutorial" user group already exists, you can run the following code sample.

In [36]:
user_group_list = client.user_groups.list_user_group()
[f for f in user_group_list if f['name'] == 'Tutorial']

[{'name': 'Tutorial',
  'createdBy': 'fa4b9fce-f6f6-4f07-bab1-5814a537c413',
  'id': '92d09891-9a8b-4b06-94d6-fe8c7c719bc3',
  'createdAt': '2023-04-06T09:26:13.392Z',
  'description': 'This user group contains all work from the TIM Tutorials.'}]

### 2. Workspaces

Now we will create a workspace in our tutorial user group. You have to set a name and specify the user group.  
You can also add a description and flag this workspace as a favorite. 

In [38]:
create_workspace_configuration = {
    "name": "Python Client",
    "description": "This workspace contains all the content created in the TIM Python Client Tutorial.",
    "userGroup": {
        "id": user_group_id
    },
    # "isFavorite": True
}

In [40]:
create_workspace_response = client.workspaces.create_workspace(configuration = create_workspace_configuration)
create_workspace_response

{'name': 'Python Client',
 'userGroup': {'name': 'Tutorial',
  'id': '92d09891-9a8b-4b06-94d6-fe8c7c719bc3'},
 'createdBy': 'fa4b9fce-f6f6-4f07-bab1-5814a537c413',
 'id': '6b851db4-6a94-42a9-85aa-ead557bc79da',
 'description': 'This workspace contains all the content created in the TIM Python Client Tutorial.',
 'createdAt': '2023-04-06T09:40:44.661Z'}

We will store the workspace information in our workflow folder and we can do the same check with the workspace as we did above with the user groups in case the workspace already exists.

In [43]:
json.dump(create_workspace_response,open("./tim_workflow/tim_workspace.json", "w"))
workspace_id = create_workspace_response['id']

workspace_list = client.workspaces.list_workspace(user_group_id = user_group_id)
[f for f in workspace_list if f['name'] == 'Python Client']

[{'name': 'Python Client',
  'userGroup': {'name': 'Tutorial',
   'id': '92d09891-9a8b-4b06-94d6-fe8c7c719bc3'},
  'createdBy': 'fa4b9fce-f6f6-4f07-bab1-5814a537c413',
  'isFavorite': False,
  'id': '6b851db4-6a94-42a9-85aa-ead557bc79da',
  'createdAt': '2023-04-06T09:40:44.661Z',
  'description': 'This workspace contains all the content created in the TIM Python Client Tutorial.'}]

## ------------------------------------------------ Next steps - Templates ------------------------------------------------

Now that we have set up our tutorial environment, we can start building models with TIM.  
For this, we can use the template notebooks listed below. You can find the notebooks in the template folder. 

### 1. Forecasting

Many different scenarios exist in time series forecasting. To accomodate these different cases, TIM offers several forecasting capabilities.  
Depending on the type of case, the phase of your analysis, the industry related to the case, ... you can choose from the different templates below.  
  
Typically, a TIM user starts with a simple forecast and backtest. Next, a more in depth analysis of the performance of TIM is executed before moving towards a full implementation of a forecasting data pipeline in production. The first 3 templates follow these scenarios.  

#### 1.1 Quick Forecast

In this template you will learn to use the quick_forecast function. This extended function is very useful for quick experimenting on a new dataset and perform a simple backtest. This will allow you to achieve the following:
- Understand TIM's forecasting capabilities
- Learn what outputs can be extracted from TIM.
- Explore your dataset and quickly find the predictive value within.

#### 1.2 Forecast Simulation

The forecast simulation template allows you to go beyond a simple backtest and learn how TIM would perform in a real production forecasting scenario.
In the notebook you can describe your simulation scenario and set up a continuous model building process. A range of simulation datetimes are chosen as the point of view from which the models are built. 
The script ensures the data situation from those datetimes are shared with TIM to avoid data leakage. From there all datetimes in the simulation range are sent to TIM in parallel to provide a quick answer to the question: What would have been the real accuracy of TIM throughout this simulation period?
From this template you will learn the following:
- Get a clear picture on the performance of TIM.
- Learn insights about the evolution over time of predictive patterns in the data.

#### 1.3 Production Forecast

The production forecast template can be used as an example to set up an automated end-to-end MLOps process with TIM. In this template, a synthetic dataset based on a real data is used to simulate a data source. This allows us to run the template and showcase the flow of data through the MLOps.  
After learning how to use this template, you can adapt the data source to a live database and immediately start the forecasting process.  
From this template you will learn the following:  
- Set up a production forecast process.
- Automate continuous model building with TIM. 

### 2. Detection

Many different scenarios exist in time series forecasting. To accomodate these different cases, TIM offers several detection capabilities.  
Depending on the type of case, the phase of your analysis, the industry related to the case, ... you can choose from the different templates below.  
  
Typically, a TIM user starts with detecting anomalies general anomalies in the entire dataset. Next, the user can focus on finding anomalies in specific variables.  
Furthermore, a more in depth analysis of the performance of TIM is executed before moving towards a full implementation of a forecasting data pipeline in production.  
The first 4 templates follow these scenarios.  

#### 2.1 System Driven Anomaly Detection

System driven anomaly detection allows the user to identify the periods in the data where anomalies across all variables occur.  
In this template you will learn:
- receive a list of automatically detected anomalies.
- to use the system driven anomaly detection method and function.
- get insights from an anomaly indicator.

#### 2.2 KPI Driven Anomaly Detection

With KPI driven anomaly detection the user can specify a specific variable and focus the anomaly detection process to an area of interest in the use case.
Use this template to learn how to:
- build a normal behavior model.
- detect anomalies in a specific variable.
- gain actionable insights from the model.

#### 2.3 Detection simulation

The detection simulation template shows what a live anomaly detection process looks like, and how live results can be leveraged to quickly alert the user of relevant anomalies. 

#### 2.4 Production Detection 

This script serves as a template to create a anomaly detection pipeline for the KPI driven approach.
Here you can learn to schedule a model rebuilding process with live detections.